In [3]:
import torch
from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification, pipeline

# === Step 1: Define paths ===
base_model = "microsoft/deberta-v3-large"         # base HF model
bin_path = "all_model/microsoft-deberta-v3-large_structure_focus_epochs4_kfold5_batch8.bin"                           # your saved .bin file
save_dir = "all_model/deberta_v3"                            # where to save Hugging Face-compatible model

# === Step 2: Rebuild model with matching config ===
config = AutoConfig.from_pretrained(base_model)
config.num_labels = 2  # binary classification

model = AutoModelForSequenceClassification.from_config(config)
state_dict = torch.load(bin_path, map_location="cpu")
model.load_state_dict(state_dict)
# === Step 3: Save the model in Hugging Face format ===
model.save_pretrained(save_dir)
# (Optional but recommended) Save tokenizer too
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.save_pretrained(save_dir)
print("saved")

/usr/local/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


saved


In [6]:
# === Step 4: Load for inference using pipeline ===
# === Step 4: Load for inference using pipeline ===
clf = pipeline(
    "text-classification",
    model=save_dir,
    tokenizer=save_dir,
    return_all_scores=True,
    device=1  # 👈 Use second GPU (cuda:1)
)

# === Step 5: Example inference ===
texts = ["when user want to escape, it escape."]
results = clf(texts)

# === Step 6: Print nicely ===
label_map = {"LABEL_0": "False", "LABEL_1": "True"}

for text, scores in zip(texts, results):
    pred = max(scores, key=lambda x: x["score"])
    print(f"\nInput: {text}")
    for s in scores:
        print(f"  {label_map[s['label']]}: {s['score']:.4f}")
    print(f"  → Prediction: {label_map[pred['label']]} (Confidence: {pred['score']:.4f})")

Device set to use cuda:1



Input: when user want to escape, it escape.
  False: 0.7711
  True: 0.2289
  → Prediction: False (Confidence: 0.7711)
